In [1]:
!pip install langchain-huggingface pinecone-client langchain langchain_groq langchain-core langchain_community langchain pymongo tiktoken langchain_pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.

**Solution 1**

**Step 1: Create an Index in Pinecone**

In [31]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

pc.create_index(
  name="ai-test",
  dimension=768,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
)


In [7]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pinecone import Pinecone, ServerlessSpec

from langchain_community.document_loaders import TextLoader
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
import os
import pinecone
# from langchain_community.vectorstores import Pinecone
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

from langchain_pinecone import PineconeVectorStore



os.environ["GROQ_API_KEY"]= "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"
os.environ["PINECONE_API_KEY"]="090c829e-d9ef-4857-9fdc-a0d4798558f4"
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

index=pc.Index("ai-test")
index_name = "ai-test"

embeddings = HuggingFaceEmbeddings()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

**Store the Embedding in Pinecone**

In [32]:
import uuid

class DocumentProcessor:
    def __init__(self):
        self.encoded_data = []

    def file_loader(self, filename):
        loader = TextLoader(filename)
        return loader.load()

    def split_embeddind_docs(self, documents, chunk_size=1000, chunk_overlap=100):
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
        docs = text_splitter.split_documents(documents)
        for document in docs:
            document.metadata['text'] = document.page_content

        for record in docs:
            doc_uuid = uuid.uuid4()
            self.encoded_data.append({
                'uuid': str(doc_uuid),
                'vector': embeddings.embed_query(record.page_content),
                'metadata': record.metadata
            })

        return self.encoded_data

    def upsertion(self, index):
        for item in self.encoded_data:
            index.upsert(
                vectors=[{
                    'id': item["uuid"],
                    'values': item['vector'],
                    'metadata': item['metadata']
                }]
            )
        return "The data has been upserted to PINECONE."

processor = DocumentProcessor()


In [34]:
filename = "/content/AI Engineer Test txt.txt"
documents = processor.file_loader(filename)
processed_data = processor.split_embeddind_docs(documents)
result = processor.upsertion(index)
print(result)

The data has been upserted to PINECONE.


**LLAMA 3.1 8B LLM with Vecor DB (RAG)**

In [43]:

docsearch = PineconeVectorStore(index_name=index_name, embedding=embeddings)
llm_grok = ChatGroq(

            groq_api_key=os.environ["GROQ_API_KEY"],

            model_name='llama-3.1-8b-instant', temperature=0.0

    )
# llama3-70b-8192

template ="""You are intelligent chatbot designed for users to ask questions about a conversation between a doctor and a patient. You will answer the relevant information from the provided transcript and answer user queries.



{context}
Question: {question}

"""
retriever=docsearch.as_retriever()

prompt = ChatPromptTemplate.from_template(template)

output_parser= StrOutputParser()
chain= ({
    "context":retriever,
    "question": RunnablePassthrough()

}
| prompt
| llm_grok
| output_parser
)


while True:
    query=input("Enter your query: ")
    if query=="":
        break
    else:

        for chunk in chain.stream(query):
            if isinstance(chunk, str):
                print(chunk, end="", flush=True)
            else:
                print(chunk.content, end="", flush=True)


Enter your query: - What doctor diagnosed?
The doctor diagnosed the patient with Generalized Anxiety Disorder (GAD).Enter your query: - What medicine doctor mentioned?
According to the transcript, the doctor mentioned prescribing the patient an SSRI (Selective Serotonin Reuptake Inhibitor) called Sertraline to help manage their anxiety symptoms.Enter your query: - Duration of medicine?
According to the transcript, the doctor mentions that it may take a few weeks to see the full effects of the prescribed medication, Sertraline (an SSRI).Enter your query: - Precautions if any?
According to the conversation between the doctor and the patient, the doctor mentioned the following precautions:

* Avoid caffeine and alcohol as they can worsen anxiety symptoms.
* Get enough sleep and try to maintain a regular sleep schedule.
* Cut back on coffee consumption (as the patient mentioned they drink a lot of coffee).

These precautions were mentioned by the doctor in the following part of the convers

In [2]:
# - What doctor diagnosed?
# - What medicine doctor mentioned?
# - Duration of medicine?
# - Precautions if any?
# - Activity if any?

**SOLUTION 2**

In [4]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
import os
import pinecone
from langchain_community.vectorstores import Pinecone
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

from langchain_pinecone import PineconeVectorStore



In [5]:
os.environ["GROQ_API_KEY"]= "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"

llm_grok = ChatGroq(

            groq_api_key=os.environ["GROQ_API_KEY"],

            model_name='llama-3.1-8b-instant', temperature=0.0

    )

template ="""You are intelligent chatbot designed for users to ask questions about a conversation between a doctor and a patient. You will answer the relevant information from the provided transcript and answer user queries.
You have the database below

Doctor (D): Good morning, how are you feeling today?
Patient (P): Good morning, Doctor. I've been feeling very anxious and stressed lately.
D: I'm sorry to hear that. Can you describe your symptoms in more detail?
P: I've been having trouble sleeping, my heart races for no reason, and I often feel
like I'm on edge. I also feel exhausted all the time.
D: It sounds like you might be experiencing symptoms of Generalized Anxiety Disorder (GAD). Have you experienced these symptoms before?
P: Yes, I've had anxiety for a few years, but it's gotten worse recently.
D: I understand. Based on your symptoms and history, I'm diagnosing you with Generalized Anxiety Disorder. We'll need to address this with a combination of medication, therapy, and lifestyle changes. Does that sound okay to you?
P: Yes, I just want to feel better.
D: For medication, I'm going to prescribe you an SSRI (Selective Serotonin Reuptake Inhibitor) called Sertraline. This should help manage your anxiety symptoms. It's important to take it as prescribed and be patient, as it may take a few weeks to see the full eects.
P: Okay, I can do that.
D: In addition to the medication, I'd like you to try some cognitive-behavioral therapy (CBT). This type of therapy can help you identify and change negative thought patterns and behaviors. I'll refer you to a therapist who specializes in CBT.
P: That sounds helpful. I've heard of CBT before.
D: Great. Now, let's talk about some exercises and lifestyle changes. Regular physical exercise can be very beneficial for reducing anxiety. Aim for at least 30 minutes of moderate exercise, like walking or yoga, most days of the week.
P: I can try to incorporate that into my routine.
D: Good. Also, practicing mindfulness or meditation daily can help reduce stress. There are
many apps and online resources that can guide you through these practices. P: I've never tried meditation, but I'm willing to give it a go.
D: Excellent. Finally, let's discuss some precautions. Avoid caeine and alcohol as they can worsen anxiety symptoms. Make sure to get enough sleep, and try to maintain a regular sleep schedule.
P: I do drink a lot of coee. I'll try to cut back.
D: It's all about making small, sustainable changes. We will monitor your progress closely and adjust the treatment plan as needed. Do you have any questions or concerns?
P: Not at the moment. Thank you, Doctor.
D: You're welcome. Remember, you're not alone in this, and we're here to support
you. I'll see you in two weeks for a follow-up. P: Thank you, Doctor. I appreciate it.




Question: {question}

"""

prompt = ChatPromptTemplate.from_template(template)

output_parser= StrOutputParser()
chain= ({
    "question": RunnablePassthrough()

}
| prompt
| llm_grok
| output_parser
)


while True:
    query=input("Enter your query: ")
    if query=="":
        break
    else:

        for chunk in chain.stream(query):
            if isinstance(chunk, str):
                print(chunk, end="", flush=True)
            else:
                print(chunk.content, end="", flush=True)


Enter your query: - What doctor diagnosed?
The doctor diagnosed the patient with Generalized Anxiety Disorder (GAD).Enter your query: - What medicine doctor mentioned?
The doctor mentioned that he is going to prescribe the patient an SSRI (Selective Serotonin Reuptake Inhibitor) called Sertraline to help manage their anxiety symptoms.Enter your query: - Duration of medicine?
According to the transcript, the doctor mentions that it may take a few weeks to see the full effects of the medication (Sertraline).Enter your query: - Precautions if any?
According to the conversation between the doctor and the patient, the precautions advised by the doctor are:

1. Avoid caffeine and alcohol as they can worsen anxiety symptoms.
2. Get enough sleep and try to maintain a regular sleep schedule.
3. Avoid excessive coffee consumption (specifically mentioned by the patient).

These precautions are mentioned by the doctor in the following lines:

"D: Excellent. Finally, let's discuss some precautions.

In [ ]:
- What doctor diagnosed?
- What medicine doctor mentioned?
- Duration of medicine?
- Precautions if any?
- Activity if any?